In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer
from datasets import Dataset
import torch
from transformers.models.bert import BertForSequenceClassification


In [2]:
data=pd.read_csv('./CleanData.csv')
data = data[ ['Description','Classification_Equipement']]
data = data.rename(columns={'Classification_Equipement': 'label'})
data.sample(15)

,Description,label
2627,Problème R.P non éffectué ===.=> Intervention ...,Mécanique
876,reparation defaut tension du convoyeur sortie ...,Mécanique
3564,Réparation défaut moteur d'extracteur poudrag...,Mécanique
2161,changement maille,Mécanique
751,changement projecteur led au niveau extérieur ...,Mécanique
5160,Correction de decor,Mécanique
4277,réglage tension tapis posimat,Mécanique
4785,changement distribiteur monte et baisse bras l...,Mécanique
2305,Décherire plastique au niveau barre de soudure,Mécanique
3070,demontage pompe et changement garniture,Mécanique


In [3]:
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 1. Load and prepare data
data = pd.read_csv('./CleanData.csv')
data = data[['Description', 'Classification_Equipement']]
data = data.rename(columns={'Classification_Equipement': 'label'})

# 2. Initialize tokenizer BEFORE defining the tokenize function
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 3. Convert text labels to numerical values
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])

# 4. Define tokenize function (now it can access the tokenizer)
def tokenize(batch):
    encoding = tokenizer(batch['Description'], padding=True, truncation=True, max_length=512)
    encoding['labels'] = batch['label']  # Add numerical labels
    return encoding

# 5. Create dataset and split
dataset = Dataset.from_pandas(data)
dataset = dataset.shuffle(seed=42)
dataset = dataset.train_test_split(test_size=0.2)

# 6. Apply tokenization
encoded_dataset = dataset.map(tokenize, batched=True)

# 7. Verify label format
print(encoded_dataset['train'][0])  # Check structure
print("Number of classes:", len(le.classes_))

# 8. Initialize model
num_labels = len(le.classes_)
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels
)

# Rest of your training code...

Map:   0%|          | 0/5218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1305 [00:00<?, ? examples/s]

{'Description': 'reparation defaut disjoncteursuite defaut demande produit', 'label': 2, 'input_ids': [101, 16360, 25879, 3258, 13366, 4887, 2102, 4487, 2015, 14339, 6593, 26744, 14663, 2063, 13366, 4887, 2102, 5157, 2063, 4013, 8566, 4183, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
num_labels = len(data['label'].unique())

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

training_args = TrainingArguments(
 output_dir="./quick_results",
    eval_steps=100,                  # Evaluate every 100 steps
    save_strategy="no",              # Disable model saving to save time
    per_device_train_batch_size=32,   # Increased batch size
    per_device_eval_batch_size=64,    # Larger eval batch
    num_train_epochs=1,               # Only 1 epoch
    max_steps=500,                    # Limit total steps
    logging_steps=50,
    learning_rate=3e-5,
    warmup_steps=50,
    fp16=True,                        # Enable mixed precision training
    gradient_accumulation_steps=2,     # Effective batch size = 64
    report_to="none",   
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

C:\Users\User\AppData\Local\Temp\ipykernel_15500\40971357.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


c:\Users\User\nlp_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


In [ ]:
trainer.evaluate()


In [ ]:
text = "CHANGEMENT POMPE A EAU"

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
prediction = outputs.logits.argmax(dim=1).item()

print(f"Predicted class: {prediction}")
